# topic and background
The main purpose of this project is to implement an application that can output github tools that correspond to the user input, in the case of the current challenge faced, and that help to deal with this challenge.

We all know that the now popular chatgpt has shown a strong knowledge base in various fields. Especially when combined with bing search, it is able to provide corresponding websites directly on demand. But chatgpt has some limitations:
1. chatGPT knowledge is trained on data before September 2021, so there is no way to provide information after that date.
2. chatGPT has no way to analyze complex logical relationships. 3.
3. ChatGPT cannot list cited sources, and its reliability is based on the reliability of the source information, which may be inherently wrong, inconsistent, or incorrect or contradictory after being combined by ChatGPT.

The above three points lead to the fact that there is no way to accomplish the purpose of this project by using chatgpt directly. We can start by looking at this challenge that we face in this presentation as follows:
“Geht es darum, die Sicherheit von Radwegen zu ermitteln, ist das Forschungspotential riesig. Ziel des SDSC-BW-Projekts war es, dieses zu erkunden. Ob der zahlreichen Problematiken gestaltete sich bereits die Erstellung eines Rahmenwerks schwierig. An erster Stelle standen die vielfältigen, teils verwirrenden Datenquellen – darunter unterschiedliche Websites, die Daten auf ihre eigene Weise abspeichern.”

Results using chatgpt:
The first is the response obtained using bing:

<img src="./images/bing_result.png" alt="feedback from bing search" width="800" height="600">

As we can see, there is no way for chaptgpt to give valid feedback, and the web page it provides is not directly related to our purpose.

The feedback obtained using the ChatGPT web page dialogue would be better:

<div style="display: inline-block;"><img src="./images/chatgpt_result.png" alt="feedback from chatgpt chatbot" width="700" height="500"></div>


Although chatgpt4 can summarize the challenges faced, the tools provided are less focused and have been developed over a longer period of time. In addition, there are many additional descriptions that make the responses look complicated.

# method
To address the problems of chatgpt, such as lack of ability to analyze complex input, complex responses, inability to provide real-time tools, and possible errors in the links provided, our core idea is to
1. decompose the requirements and only ask simple questions to gpt at a time
2. restrict the output so that the output is brief and linked to the topic
3. use github api to get the latest github, to ensure the popularity and effectiveness of the tool

Here we start to show how we do it, first of all, load the required packages

In [1]:
from git_request import search_top_starred_repositories
from gpt_request import get_response_from_chatgpt_with_context

import os
import pandas as pd
import requests

Where git_request and gpt_request are both custom packages for wanting github server to request github list and asking openai to request chatgpt service respectively.
pandas is a data analysis tool and request is a network access tool

In [2]:
use_case = input("Please input the use case: ")

# get the theme from the user case
prompt = f"What is the theme studied in the following use case, please answer only with a keyword less then 20 letter: {use_case}"
context = []
response, context = get_response_from_chatgpt_with_context(prompt, context)
print(f"\nThe theme of the use case is: {response}\n")


Please input the use case:  Geht es darum, die Sicherheit von Radwegen zu ermitteln, ist das Forschungspotential riesig. Ziel des SDSC-BW-Projekts war es, dieses zu erkunden. Ob der zahlreichen Problematiken gestaltete sich bereits die Erstellung eines Rahmenwerks schwierig. An erster Stelle standen die vielfältigen, teils verwirrenden Datenquellen – darunter unterschiedliche Websites, die Daten auf ihre eigene Weise abspeichern.



The theme of the use case is: Radwegsicherheit



First we let gpt analyze the topic corresponding to the input, and we saw that gpt was able to do this well. Here, to limit the brevity of the answer, we restricted the output of gpt by adding the requirement

In [3]:
# get the challenge from the user case
prompt = f"According to the given description, what is the main problem faced by this study, please answer with 3 keywords and without explanation"
response, context = get_response_from_chatgpt_with_context(prompt, context)
print(f"The main challenge facing are: \n{response}\n")

The main challenge facing are: 
Multiple data sources, confusing data, complex framework.



We then asked for a summary of the three challenges faced given the topic. Because this challenge is to be applied to a subsequent github search, we asked gpt to respond using keywords and without giving an explanation.

In [4]:
# ask for keywords for python tools
prompt = f"I want to search for python tools for the above problem by keywords, what keywords should I use, please give me 3 suggestions and speperate them with semicolon, without explanation"
response, context = get_response_from_chatgpt_with_context(prompt, context)
keywords = response.split(";")

# list the advised git repos
for keyword in keywords:
    prompt = f"Please explain why data {keyword} is needed in the context of the use case above, and please answer in less than 50 words"
    response, context = get_response_from_chatgpt_with_context(prompt, context)
    print(response)

    git_urls, readme_urls = search_top_starred_repositories(keyword+' python')
    if git_urls is not None:
        print("For this, we recommend the following tools:")
        for git_url, readme_url in zip(git_urls, readme_urls):
            print("Repository URL:", git_url)
            print("README URL:", readme_url)
        print('-'*50)

Data scraping is needed to extract and collect data from various online sources, including websites, in order to obtain a comprehensive dataset for analyzing and assessing the safety of bike lanes.
For this, we recommend the following tools:
Repository URL: https://github.com/khuyentran1401/Data-science
README URL: https://github.com/khuyentran1401/Data-science/blob/master/README.md
Repository URL: https://github.com/stanfordjournalism/search-script-scrape
README URL: https://github.com/stanfordjournalism/search-script-scrape/blob/master/README.md
Repository URL: https://github.com/hhursev/recipe-scrapers
README URL: https://github.com/hhursev/recipe-scrapers/blob/master/README.md
Repository URL: https://github.com/scrapy/parsel
README URL: https://github.com/scrapy/parsel/blob/master/README.md
Repository URL: https://github.com/damklis/DataEngineeringProject
README URL: https://github.com/damklis/DataEngineeringProject/blob/master/README.md
--------------------------------------------

Based on each keyword provided above, we find matching tools from the github server and return the top 5 results ranked according to how widely they are used. Since the results are provided through the github server, this ensures the reliability of the results. In addition, because we match by summarized keywords, the results obtained are closely related to our research topic.

The keyword search is based on the following three contents:
1. the name and description of the repository
2. source code and file contents
3. issue

This ensures that the repository can be searched even if the repository owner's keywords are set incorrectly, and because the keywords are summarized by chatgpt, this ensures that the keywords obtained do not contain any out-of-the-ordinary words.

# localization
Of course, there are disadvantages to the above operation. For example, the keywords must appear in the content of the repository; they must be connected to the network.

The first problem can be solved by word embedding. By word embedding the keywords and the readme file of the repository, we can find the matching repository by comparing the similarity of the two embeddings. The second problem can be solved by local deployment of gpt. openai's chatgpt-4 is not yet open source, but there are already many open source alternatives. We can achieve our goal by deploying these alternatives locally.

To do this we first load two custom methods

In [5]:
from generate_local_github_database import download_and_save_git_stared_reposiories
from git_request import search_top_related_local_repositories

download_and_save_git_stared_reposiories implements the operation of downloading the specified repository information and saving it locally. search_top_related_local_repositories implements the deployment of local gpt, the implementation of word embedding and the search of local databases.

In [6]:
# ask for keywords for python tools
prompt = f"I want to search for python tools for the above problem by keywords, what keywords should I use, please give me 3 suggestions and speperate them with semicolon, without explanation"
response, context = get_response_from_chatgpt_with_context(prompt, context)
keywords = response.split(";")

# list the advised git repos
for keyword in keywords:
    prompt = f"Please explain why data {keyword} is needed in the context of the use case above, and please answer in less than 50 words"
    response, context = get_response_from_chatgpt_with_context(prompt, context)
    print(response)

    git_urls, readme_urls = search_top_related_local_repositories(keyword, database_path = './data/repositories.csv')
    if git_urls is not None:
        print("For this, we recommend the following tools:")
        for git_url, readme_url in zip(git_urls, readme_urls):
            print("Repository URL:", git_url)
            print("README URL:", readme_url)
        print('-'*50)


Web scraping is necessary to collect data from various online sources, including websites, for analyzing the safety of bike lanes. It automates the data collection process and creates a comprehensive dataset for further analysis.


/pfs/data5/home/kit/tm/hj7422/conda/envs/vicuna/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


Using embedded DuckDB without persistence: data will be transient
Unable to connect optimized C data functions [No module named 'clickhouse_connect.driverc.buffer'], falling back to pure Python
Unable to connect ClickHouse Connect C to Numpy API [No module named 'clickhouse_connect.driverc.npconv'], falling back to pure Python


For this, we recommend the following tools:
Repository URL: https://github.com/RaRe-Technologies/gensim
README URL: https://github.com/RaRe-Technologies/gensim/blob/master/README.md
Repository URL: https://github.com/explosion/sense2vec
README URL: https://github.com/explosion/sense2vec/blob/master/README.md
Repository URL: https://github.com/BrikerMan/Kashgari
README URL: https://github.com/BrikerMan/Kashgari/blob/master/README.md
Repository URL: https://github.com/gnes-ai/gnes
README URL: https://github.com/gnes-ai/gnes/blob/master/README.md
Repository URL: https://github.com/github/semantic
README URL: https://github.com/github/semantic/blob/master/README.md
--------------------------------------------------
Data wrangling is necessary to clean, transform, and harmonize the extracted dataset from different online sources. It enables data cleaning, data manipulation, and integration to ensure consistency, quality, and accuracy of the data used for analyzing the safety of bike lanes.


Using embedded DuckDB without persistence: data will be transient


max_seq_length  512
For this, we recommend the following tools:
Repository URL: https://github.com/cpitclaudel/dBoost
README URL: https://github.com/cpitclaudel/dBoost/blob/master/README.md
Repository URL: https://github.com/amundsen-io/amundsen
README URL: https://github.com/amundsen-io/amundsen/blob/master/README.md
Repository URL: https://github.com/doccano/doccano
README URL: https://github.com/doccano/doccano/blob/master/README.md
Repository URL: https://github.com/dmlc/gluon-nlp
README URL: https://github.com/dmlc/gluon-nlp/blob/master/README.md
Repository URL: https://github.com/flairNLP/flair
README URL: https://github.com/flairNLP/flair/blob/master/README.md
--------------------------------------------------
Data visualization is necessary to present the analyzed data in a clear and visually appealing format. It helps to identify trends, patterns, and insight to stakeholders, policymakers, and residents, promoting data-driven decision-making to improve the safety of bike lanes

Using embedded DuckDB without persistence: data will be transient


max_seq_length  512
For this, we recommend the following tools:
Repository URL: https://github.com/PAIR-code/facets
README URL: https://github.com/PAIR-code/facets/blob/master/README.md
Repository URL: https://github.com/amundsen-io/amundsen
README URL: https://github.com/amundsen-io/amundsen/blob/master/README.md
Repository URL: https://github.com/bokeh/bokeh
README URL: https://github.com/bokeh/bokeh/blob/master/README.md
Repository URL: https://github.com/raghakot/keras-vis
README URL: https://github.com/raghakot/keras-vis/blob/master/README.md
Repository URL: https://github.com/yosinski/deep-visualization-toolbox
README URL: https://github.com/yosinski/deep-visualization-toolbox/blob/master/README.md
--------------------------------------------------


We can see that it works well locally as well. Also, since the test database we applied contains only two hundred different repositories and is more centralized, the results do not look as good as the online ones. But this can be solved by increasing the local dataset.